<div style="background-color: steelblue; padding: 10px; border-radius: 5px;">
    <p style="margin: 10;"></p>
    <h1 style="text-align: center; margin: 0; font-weight: bold; color: white;">SWOT Ocean Model Intercomparison Project : EORCA12-ERA5v1 Test Case</h1>
    <p style="margin: 10;"></p>
</div>



## 📦**Imports** 

In [1]:
%%time

##################################
#Imports

from datetime import datetime
import s3fs
import xarray as xr
import pyinterp
from widetrax import DataPreprocessing as dp
from widetrax import Spectra as sp
import numpy as np
import json
from watermark import watermark
import time
import platform

CPU times: user 3.06 s, sys: 1.2 s, total: 4.26 s
Wall time: 1.64 s


## 🔍 **Check the S3 Endpoint**  

In [2]:
s3_folder ="s3://project-moi-swot-omip/GLO12V4" # Do not write /!!!!!!
endpoint_url =  "https://minio.dive.edito.eu/"
fs = s3fs.S3FileSystem(anon=True,endpoint_url=endpoint_url)
# List contents of the bucket
bucket_name = 's3://project-moi-swot-omip/GLO36V1'
contents = fs.ls(bucket_name)
print("Bucket contents:")
for item in contents:
    print(item)

Bucket contents:
project-moi-swot-omip/GLO36V1/cycle_.keep
project-moi-swot-omip/GLO36V1/cycle_008
project-moi-swot-omip/GLO36V1/cycle_009
project-moi-swot-omip/GLO36V1/cycle_010
project-moi-swot-omip/GLO36V1/cycle_011
project-moi-swot-omip/GLO36V1/cycle_012
project-moi-swot-omip/GLO36V1/cycle_013
project-moi-swot-omip/GLO36V1/cycle_014
project-moi-swot-omip/GLO36V1/cycle_015
project-moi-swot-omip/GLO36V1/cycle_016
project-moi-swot-omip/GLO36V1/cycle_017
project-moi-swot-omip/GLO36V1/cycle_018
project-moi-swot-omip/GLO36V1/cycle_019
project-moi-swot-omip/GLO36V1/cycle_020
project-moi-swot-omip/GLO36V1/cycle_021
project-moi-swot-omip/GLO36V1/cycle_022
project-moi-swot-omip/GLO36V1/cycle_023
project-moi-swot-omip/GLO36V1/cycle_024
project-moi-swot-omip/GLO36V1/cycle_025
project-moi-swot-omip/GLO36V1/cycle_026


## 🧮 **Required variables**

In [3]:
# REGION A

name_region="Region_B"
season="JAS"

if name_region=="Region_A":
    lon_min = 360-70,
    lon_max = 360-60,
    lat_min = 32,
    lat_max = 42,
elif name_region=="Region_B":
    lon_min = 360-50,
    lon_max = 360-40,
    lat_min = 17,
    lat_max = 27,

area = [lon_min, lat_min, lon_max, lat_max]

if season=="JFM":    
    start_date = "01012024" # "DDMMYYYY"
    end_date ="31032024"
elif season=="JAS":
    start_date = "01072024" # "DDMMYYYY"
    end_date ="30092024"

In [4]:
end_date

'30092024'

## 🔄 **Identify the cycle numbers within the specified time range** 

In [4]:
if season =="JFM":
    file_path = "https://minio.lab.dive.edito.eu/project-meom-ige/cycles_periods.csv" # works only for winter period
elif season =="JAS":
    file_path = "time_ranges.csv"  # for summer

matching_cycles = dp.get_matching_cycles(file_path, start_date, end_date)

def formater_numeros_concis(liste_numeros):
  return [str(numero).zfill(3) for numero in liste_numeros]
    
matching_cycles = formater_numeros_concis(matching_cycles)
matching_cycles


['017', '018', '019', '020', '021']

## 📊 **Load datasets and compute Wavenumber spectra**

In [5]:
%%time

# Load datasets into a dictionary
datasets_dict = dp.read_swot_ncfiles_S3subfolders(
    s3_folder,
    matching_cycles,
    endpoint_url,
    area)

17_005_ included.
17_018_ included.
17_033_ included.
17_046_ included.
17_074_ included.
17_102_ included.
17_199_ included.
17_227_ included.
17_255_ included.
17_268_ included.
17_283_ included.
17_296_ included.
17_311_ included.
17_324_ included.
17_352_ included.
17_380_ included.
17_505_ included.
17_533_ included.
17_546_ included.
17_561_ included.
17_574_ included.
18_005_ included.
18_018_ included.
18_033_ included.
18_046_ included.
18_074_ included.
18_102_ included.
18_199_ included.
18_227_ included.
18_255_ included.
18_268_ included.
18_283_ included.
18_296_ included.
18_311_ included.
18_324_ included.
18_352_ included.
18_380_ included.
18_505_ included.
18_533_ included.
18_546_ included.
18_561_ included.
18_574_ included.
19_005_ included.
19_018_ included.
19_033_ included.
19_046_ included.
19_074_ included.
19_102_ included.
19_199_ included.
19_227_ included.
19_255_ included.
19_268_ included.
19_283_ included.
19_296_ included.
19_311_ included.
19_324_ in

In [6]:
%%time 
has_converged, filled_datasets = dp.fill_nan(datasets_dict, varname = "ssh")

CPU times: user 2.75 s, sys: 5.93 s, total: 8.68 s
Wall time: 3.76 s


In [7]:
%%time
segments_dict = sp.retrieve_segments(filled_datasets,FileType = "NetCDF",namevar="ssh")

/opt/anaconda3/envs/myenv/lib/python3.12/site-packages/widetrax/Spectra.py:43: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  for col in range(dataset.dims['num_pixels']):


CPU times: user 274 ms, sys: 5.34 ms, total: 280 ms
Wall time: 278 ms


In [8]:
%%time
psd_dict, freqs_dict = sp.calculate_psd(segments_dict)
# Calculate PSD Mean
psd_mean, freqs_mean = sp.psd_mean_and_freq(psd_dict,freqs_dict)

CPU times: user 702 ms, sys: 473 ms, total: 1.18 s
Wall time: 4.62 s


In [9]:
psd_mean

array([1.00336418e-01, 5.89500229e-01, 4.19924270e-01, 2.54119823e-01,
       9.62950234e-02, 4.73910880e-02, 3.03424533e-02, 1.81197329e-02,
       1.05849287e-02, 6.33282993e-03, 3.95946551e-03, 2.43525617e-03,
       1.73106268e-03, 1.31487160e-03, 7.78208938e-04, 4.41067169e-04,
       3.47258941e-04, 2.64369145e-04, 1.86681560e-04, 1.38310054e-04,
       1.06101017e-04, 7.98704324e-05, 5.58013652e-05, 4.11013636e-05,
       3.22821874e-05, 2.35777752e-05, 1.84259751e-05, 1.51664269e-05,
       1.27012935e-05, 1.01771159e-05, 7.72882914e-06, 5.57223409e-06,
       4.76960833e-06, 3.64903561e-06, 2.86636682e-06, 2.41204027e-06,
       1.95461640e-06, 1.64687072e-06, 1.36150658e-06, 1.05025525e-06,
       8.83206526e-07, 7.57359161e-07, 6.33965009e-07, 5.53777970e-07,
       4.94722960e-07, 4.20008481e-07, 3.65660103e-07, 3.31479072e-07,
       2.93578783e-07, 2.52131437e-07, 2.18603017e-07, 2.02322432e-07,
       1.89354426e-07, 1.72571002e-07, 1.68125901e-07, 1.58073273e-07,
      

## 💾 **Save Results and Information in JSON File**

In [10]:
# Infos
nom_region = name_region
model = "GLO12V4"
name_season = season
freqs_mean = np.array(freqs_mean)
psd_mean = np.array(psd_mean)
todays_Date = datetime.now()
date_creation = todays_Date.isoformat() #ISO 8601 format
packages_versions = watermark(packages="numpy,widetrax,xarray,s3fs,pyinterp,datetime,json,platform,time")
machine_info = {
    "system": platform.system(),
    "node_name": platform.node(),
    "release": platform.release(),
    "version": platform.version(),
    "machine": platform.machine(),
    "processor": platform.processor(),
    "python_version": platform.python_version()
}

# JSON file structure
donnees = {
    "region": nom_region,
    "model": model,
    "lon_min": lon_min,
    "lon_max": lon_max,
    "lat_min": lat_min,
    "lat_max": lat_max,
    "start_date": datetime.strptime(start_date, "%d%m%Y").strftime("%Y-%m-%d"),
    "end_date": datetime.strptime(end_date, "%d%m%Y").strftime("%Y-%m-%d"),
    "packages_versions": packages_versions,
    "date_creation": date_creation,
    "login_creator": platform.node(),
    "Frequency": freqs_mean.tolist(),
    "Spectra": psd_mean.tolist(),
    "machine_info": machine_info
}

# Save on JSON file
with open(str(nom_region)+"_"+str(name_season)+"_"+str(model)+".json", "w", encoding="utf-8") as f:
    json.dump(donnees, f, ensure_ascii=False, indent=4)
#with open("OMIP_"+str(model)+"_"+str(nom_region)+"_"+str(name_season)+".json", "w", encoding="utf-8") as f:
  #  json.dump(donnees, f, ensure_ascii=False, indent=4)

#<region>-<period>-<data-source>.json
print("JSON file created")

JSON file created


## 📤 **Export Results to the S3 Endpoint** 

In [11]:
fs = s3fs.S3FileSystem( anon=True, endpoint_url="https://minio.lab.dive.edito.eu", use_ssl=False ) 

In [12]:
json_file = str(nom_region)+"_"+str(name_season)+"_"+str(model)+".json"
fs.put(json_file, "project-meom-ige/OMIP/")

[None]

In [16]:
fs